In [ ]:
#------ Prologue
import postgkyl as pg
import numpy as np
#import pylab as plt
#import matplotlib as mpl
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (6,4)
import matplotlib.cm as cm
import os, fnmatch, re, sys
#------ add personnal python utilities
home_dir = os.path.expanduser("~")
sys.path.append(home_dir+'/personal_gkyl_scripts/python_utilities')
from classes import Simulation, Species
from utils import *
#----- some parameters of the simulation
#----- physical quantities
simulation = Simulation()
simulation.set_phys_param(
    B_axis = 1.4,           # Magnetic field at magnetic axis [T]
    eps0 = 8.854e-12,       # Vacuum permittivity [F/m]
    eV = 1.602e-19,         # Elementary charge [C]
    mp = 1.673e-27,         # Proton mass [kg]
    me = 9.109e-31,         # Electron mass [kg]
)
simulation.set_geom_param(
    R_axis      = 0.8727315068,         # Magnetic axis major radius
    Z_axis      = 0.1414361745,         # Magnetic axis height
    R_LCFSmid   = 1.0968432365089495,   # Major radius of LCFS at the midplane
    a_shift     = 0.25,                 # Parameter in Shafranov shift
    kappa       = 1.45,                 # Elongation factor
    delta       = 0.35,                 # Triangularity factor
    q0          = None,                 # Safety factor
    x_LCFS      = 0.04                  # position of the LCFS (x_inner)
)

simulation.set_data_param(
    expdatadir ='/home/ah1032/amd/g0_sim/diagnostics/gk_tcv/tcv_exp_data/',
    g0simdir   = home_dir+'/g0_sim/',
    simname    = '48x32x16x16x6_nunifvg',
    simdir     = 'gk_tcv_posD_iwl_3x2v_AH/',
    fileprefix = 'gk_tcv_posD_iwl_3x2v_D02',
    wkdir      = 'wk/'
)

# Define the species
ion = Species(name='ion',
              m=2.01410177811*simulation.phys_param.mp, # Ion mass
              q=simulation.phys_param.eV,               # Ion charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
elc = Species(name='elc',
              m=simulation.phys_param.me, 
              q=-simulation.phys_param.eV, # Electron charge [C]
              T0=100*simulation.phys_param.eV, 
              n0=2.0e19)
simulation.add_species(ion)
simulation.add_species(elc)
# Check available frames
tfs = find_available_frames(simulation,'field')
print(tfs)

#---- Set up normalization and units
simulation.normalize('t','mus')
simulation.normalize('x','LCFS')
simulation.normalize('y','Larmor radius')
simulation.normalize('fluid velocities','thermal velocity')
simulation.normalize('temperatures','eV')
simulation.normalize('pressures','beta')

In [ ]:
# Load metric coefficients, magnetic field
simulation.geom_param.load_metric(simulation.data_param.fileprefix)

plot_domain(simulation.geom_param)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quad

# Parameters
Rdim = 1.7  # [m]
Zdim = 3.2  # [m]
Z_axis = 0.013055028  # [m]
R_axis = 1.65  # [m]
R_LCFSmid = 2.17  # [m]
Rmid_min = R_LCFSmid  # Minimum midplane major radius of simulation box [m].
Rmid_max = 2.35  # Maximum midplane major radius of simulation box [m].
R0 = 0.5 * (Rmid_min + Rmid_max)  # Major radius of the simulation box [m].
a_mid = R_LCFSmid - R_axis  # Minor radius at outboard midplane [m].
r0 = R0 - R_axis  # Minor radius of the simulation box [m].
B_axis = 3.0  # Example value for B-axis [T]
B0 = B_axis * (R_axis / R0)  # Magnetic field magnitude in the simulation box [T].

qAxis = 0.7
qSep = 3.0  # Safety factor at the separatrix.
kappa = 1/1.5  # Elongation
delta = 0.2  # Triangularity

# Define functions
def r_x(x):
    return x + a_mid  # Minor radius given x

def qprofile(r):
    return qAxis + (qSep - qAxis) * (r / a_mid) ** 2  # Magnetic safety profile

def s(r):
    return 2 * r**2 / (qprofile(r) * a_mid)**2 * (qSep - qAxis)

# Define Miller geometry functions for R and Z
def R_geom(r, theta):
    return (R_axis + r * np.cos(theta) +
            r * (kappa - 1) / (kappa + 1) * (qprofile(r) * s(r) + 6 * qAxis) / 
            (qSep * s(a_mid) + 6 * qAxis) * np.cos(theta) + 
            delta * r**2 / (4 * a_mid) * (qprofile(r) * s(r) + 4 * qAxis) / 
            (qSep * s(a_mid) + 4 * qAxis) * np.cos(2 * theta))

def Z_geom(r, theta):
    return (r * np.sin(theta) - 
            r * (kappa - 1) / (kappa + 1) * (qprofile(r) * s(r) + 6 * qAxis) / 
            (qSep * s(a_mid) + 6 * qAxis) * np.sin(theta) -
            delta * r**2 / (4 * a_mid) * (qprofile(r) * s(r) + 4 * qAxis) / 
            (qSep * s(a_mid) + 4 * qAxis) * np.sin(2 * theta))

def Bphi(R):
    return B0 * R0 / R  # Toroidal magnetic field

# Plotting R(r, theta) and Z(r, theta) on a meshgrid
r_vals = np.linspace(0.1, a_mid, 5)
theta_vals = np.linspace(0, 2 * np.pi, 100)
r_grid, theta_grid = np.meshgrid(r_vals, theta_vals)

R_values = R_geom(r_grid, theta_grid)
Z_values = Z_geom(r_grid, theta_grid)

plot_domain(simulation.geom_param)
plt.plot(R_values,Z_values,'-b')
plt.axis('equal')
plt.xlabel('R (m)')
plt.ylabel('Z (m)')
plt.show()